<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Regresión polinomial

Ejemplo de regresión polinomial<br>
v2.0

### Objetivos:
*   Comprender como funciona la regresión polinomial en un ejemplo real.
*   Evaluar el resultado de una regresión polinomial considerando el grado y el error mínimo.
* Realizar una predición con un nuevo valor.

In [ ]:
#Librerias a implementar
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

### Código de descarga del dataset

In [ ]:
if os.access('FuelConsumptionCo2.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv > FuelConsumptionCo2.csv
    else:
        !wget FuelConsumptionCo2.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv

### `FuelConsumption.csv`:
El dataset **`FuelConsumption.csv`** contiene el consumo específico por tipo de vehículo y la emisión estimado de dioxido de carbono (Co2) de nuevos vehículos que son venidos en canada.<br> [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

- **MODELYEAR** --> ejemplo 2014
- **MAKE** --> ejemplo Acura
- **MODEL** --> ejemplo ILX
- **VEHICLE CLASS** --> ejemplo SUV
- **ENGINE SIZE** --> ejemplo 4.7
- **CYLINDERS** --> ejemplo 6
- **TRANSMISSION** --> ejemplo A6
- **FUEL CONSUMPTION in CITY(L/100 km)** --> ejemplo 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** --> ejemplo 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** --> ejemplo 9.2
- **CO2 EMISSIONS (g/km)** --> ejemplo 182

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Una vez descargado el archivo en Colab.
# Leerlo con Pandas y el método read_csv
# Una vez extraida toda la información se almacena en df
# A partir de df y el método describe(), mostrará la descripción estadistica básica del archivo que se guardará en des
# Crear una fila nueva llamada Nan en el DataFrame  des,
# que indica la cantidad de datos tipo Nan que tiene cada columna.
# Para crear una nueva fila, se utilizará el operador loc, donde se indica el nombre
# de la nueva fila y con que valores se completará.
# La información será de los datos faltantes df.isna().sum()
# Crear una fila nueva llamada %Nan en el DataFrame des,
# Esta fila se completará con los porcentajes de Nan encontrados en cada columna.
df = pd.read_csv("FuelConsumptionCo2.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
# Muestra las 5 primeras filas del DataFrame df
df.head()

## Fin de la limpieza
Se finalizó la limpieza, no hay datos mal cargados o incompletos en este dataset

In [ ]:
# ¿Cuántos datos quedaron para analizar?
# (filas, columnas)
# Al ser shape[0] accede a la información de las filas
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Analizaremos como se compartan algunos atributos de entrada contra el objetivo (las emisiones, el Co2)
# sns alias de Seaborn
# se empleará el método pairplot, que muestra las relaciones de varias columnas con 
# respecto a una columna de referencia
# En este caso, con respecto a CO2EMISSIONS
# diag_kind=None, parámetro que indica el tipo de trama para las subtramas diagonales. En este caso, ninguna trama
# height=5, altura

pp = sns.pairplot(data=df, x_vars=['CYLINDERS', 'ENGINESIZE', 'FUELCONSUMPTION_COMB'], y_vars=['CO2EMISSIONS'], diag_kind=None, height=5)
# Muestra la figura
plt.show()


Se puede observar que todas tienen una relación lineal con le emisión de CO2, pero para este ejemplo nos quedaremos con las cilindradas del motor para evaluar contra el CO2. Queda en la voluntad del alumno de seguir experimentando con otros features

In [ ]:
# Analizamos la distribución de la variable que se desea analizar

# Se crea el espacio para dibujar con fig = plt.figure(figsize=(16, 9))
# Se crea el espacio para el gráfico
# sns alias Seaborn
# accede al método boxplot(), donde se le indica los valores a representar en el eje de las x, en este caso, 
# los valores de la columna m2
# ax=ax, es un objeto Axes sobre el que se dibuja el gráfico; de lo contrario, utiliza los ejes actuales.
# ax.grid('dashed') para ver la grilla de fondo especificando el color
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.boxplot(x=df['CYLINDERS'], ax=ax, color='darkCyan')
ax.grid('dashed')

Se puede observar que está centrada a la izquierda la distribución y no hay outliers, por lo que no habrá problema. El hecho de que la distribución esté marcada a la izquierda querra decir que hay menos datos para analizar de alta cilindrada.

In [ ]:
# Se crea el espacio para dibujar con fig = plt.figure(figsize=(16, 9))
# Se crea el espacio para el gráfico
# Se representa un grafico de dispersión con ax = sns.scatterplot()
# Se especifica los valores para x e y
# ax=ax, es un objeto Axes sobre el que se dibuja el gráfico; de lo contrario, utiliza los ejes actuales.
# ax.grid('dashed') para ver la grilla de fondo especificando el color
fig = plt.figure()
ax = fig.add_subplot()
ax = sns.scatterplot(x=df['CYLINDERS'], y=df['CO2EMISSIONS'], color='darkRed', ax=ax)
ax.grid('dashed')
plt.show()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
# Para los valores de X se emplea doble corchete porque necesita la matriz en dos dimensiones
X = df[['CYLINDERS']].values

# Para los valores de y se emplea un corchete porque necesita la matriz en una dimensiones
y = df['CO2EMISSIONS'].values

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
# Se importa la herramienta de sklearn.model_selectionl como train_test_split
from sklearn.model_selection import train_test_split

# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
# Ojo! Los dataset de train y test son array numpy
# Se importa la herramienta de la libreria  train_test_split()
# Necesita los valores de X e y
# test_size=0.3, permite indicar el porcentaje de valores para evaluar, equivalente a un 30%
# random_state=42,  es un número fijo que utilizan comunmente en documentación, significa que para cada ejecución del algoritmo se genere nuevos valores aleatorios
# y los conjuntos de datos de entrenamiento y pruebas serán diferentes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Crear un modelo de regresión polinominal

In [ ]:
# Se importa el algoritmo de sklearn.preprocessing  como PolynomialFeatures
# Se importa la métrica de sklearn.metrics como mean_squared_error(MSE)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Se tendrán dos variables con listas vacías que almacenarán
# Una mse_list, los errores
# grados_list, almacenará los grados del polinomio de cada entrenamiento 
mse_list = []
grados_list = []

# El bucle iterará 9 veces, para degree 1,2,3,4,5,6,7,8,9
for degree in range(1, 10):

    # Se crea el objeto polynomial_features a partir de la clase  PolynomialFeatures()
    # y se indica la variable degree    
    poly = PolynomialFeatures(degree=degree)
    
    # Se aplica el método fit_transform indicando los X para entrenar y evaluar
    # haciendo un ajuste con reshape
    X_train_poly = poly.fit_transform(X_train.reshape(-1, 1))
    X_test_poly = poly.fit_transform(X_test.reshape(-1, 1))

    # Se aplica una regresión lineal una vez conseguido los valores para X_train y X_test del polinomio 
    # Se crea el objeto de regresión lineal
    # Se aplica el método fin con los X del polinomio
    # Luego se hace el predict
    lr = LinearRegression()
    lr.fit(X_train_poly, y_train)
    y_hat = lr.predict(X_test_poly)

    # Se calcula el error (MSE)
    mse = mean_squared_error(y_test, y_hat)

    # El error en cada iteración se va guardando en la lista mse_list
    mse_list.append(mse)

    # Al igual que el grado del polinomio
    grados_list.append(degree)

    # Asi como también, en cada iteración se hace un print del grado y error.
    print(f"Gradio de polinomio {degree}, error: {mse}") # print del error

In [ ]:
# Representación gráfica de linea
# De los grados con respecto al error
plt.title("Nivel óptimo de complejidad del modelo")
plt.plot(grados_list, mse_list, label="test")

# Identificación del eje y 
plt.ylabel("Error")
plt.show()

# Print de error más bajo y el grado que corresponde a ese error
print('Error mínimo:', min(mse_list))
print('Nivel óptimo:', mse_list.index(min(mse_list))+1)

Construir el modelo que mejor ajuste al problema

In [ ]:
# Se importa el algoritmo de  sklearn.preprocessing  como PolynomialFeatures
# Se importa el normalizador como MinMaxScaler de sklearn.preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Se crea el objeto poly
poly = PolynomialFeatures(degree=3)

# Se aplica fit_transform con el grado con menor error
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

# Una vez que se tienen los valores de X del polinomio identificado 
# Se aplica una regresiín lineal con los nuevos valore de X
lr = LinearRegression()
lr.fit(X_train_poly, y_train)
y_hat = lr.predict(X_test_poly)

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Es un coeficiente de determinación, determina la capacidad de un modelo para predecir futuros resultados. 
# El mejor resultado posible es 1.0
from sklearn.metrics import r2_score
score = r2_score(y_test, y_hat)
print(f"Coeficiente de determinación: {score:.2f}")

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
# Modelo de Regresión Polinomial
# El array debe tener el valor de los cilindros
emisiones = np.array([[6]])

#Transforma el nuevo valor a polinomio
emisiones_poly = poly.fit_transform(emisiones)
y_hat_prueba = lr.predict(emisiones_poly)
y_hat_prueba

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

En este ejemplo emisiones de CO2 se puede realizar todos los tipos de regresioens estudiadas hasta el momento, en este caso solo se comparó contra la cilindrada utilizando una regresión polinomial